In [4]:
using Distributions
using Statistics
include("funtions.jl")
using BenchmarkTools

In [2]:
function PSO(d, l, u, Part_N, Max_iter)
    
  #Proceso de inicialización de PSO
  x = l' .+ rand(Uniform(0,1), Part_N, d) .* (u - l)'
  #Evalua la función objetivo
  obj_func = [rosenbrock(x[i, :], d) for i=1:Part_N]
  #Obtiene el mejor valor global(mínimo)
  glob_opt = minimum(obj_func)
  ind = argmin(obj_func)
  #Vector de valores optimos
  G_opt = reshape(x[ind, :], 1, d) * ones(d, Part_N)
  Mejor_pos = x[ind, :]
  #Mejor local para cada partícula
  Loc_opt = x
    
  #Inicializa velocidades
  v = zeros(Part_N, d)

  #Vector para poder evaluar las nuevas posiciones
  #en la función objetivo
  Nva_obj_func = zeros(1, Part_N)
  Evol_func_obj = zeros(1, Max_iter)
    
  #Inicia proceso de optimización PSO
  t = 1
  #Criterio de paro
  while t < Max_iter
    #Calcula la nueva velocidad
    v = v .+ rand(Uniform(0,1), Part_N, d) .* (Loc_opt - x) .+ rand(d)' .* (G_opt' .- x)
    #Calcula nueva posición
    x = x .+ v
    #Se verifica que las partículas no se salan de los límites
    for i=1:Part_N
      if x[i, :] > u
        x[i, :] = u
      elseif x[i, :] < l
        x[i, :] = l
      end
      #Se evalúan las nuevas posiciones en la funcion objetivo
      Nva_obj_func[i] = rosenbrock(x[i, :], d)
      #Se verifica si se actualizaron los óptimos locales
      if Nva_obj_func[i] < obj_func[i]
        #Actualiza óptimo local
        Loc_opt[i, :] = x[i, :]
        #Actualiza función objetivo
        obj_func[i] = Nva_obj_func[i]
      end
    end
    #Obtiene el mejor valor global
    Nvo_glob_opt = minimum(obj_func)
    ind = argmin(obj_func)
    #Se verifica si se actualiza el óptimo global
    if Nvo_glob_opt < glob_opt
      glob_opt = Nvo_glob_opt
      #Obtenemos los valores óptimos en cada dimensión
      G_opt[:] = reshape(x[ind, :], 1, d) * ones(d, Part_N)
      Mejor_pos = x[ind, :]
    end
    #Almacena los valores de función objetivo en cada iteración
    Evol_func_obj[t] = glob_opt
    #Incrementa iteraciones
    t = t + 1
  end
  
  return Mejor_pos
end

PSO (generic function with 1 method)

In [3]:
PSO(2,[-5,-5],[10,10],100,100)

2-element Vector{Float64}:
 0.9906411003995645
 0.9671665618817766

In [5]:
@benchmark PSO(2,[-5,-5],[10,10],100,100)

BenchmarkTools.Trial: 478 samples with 1 evaluation.
 Range (min … max):   9.800 ms …  14.259 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     10.225 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   10.463 ms ± 671.737 μs  ┊ GC (mean ± σ):  1.79% ± 4.86%

       ▃▇█▇▆▆▄▄▄▂▂                                   ▂▁         
  ▄▄▁▁▁███████████▄▄▁▄▄▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▄▁▁▄▁▁▁▁▁▆▅▆▇██▄██▄▄▄▄ ▇
  9.8 ms        Histogram: log(frequency) by time      12.6 ms <

 Memory estimate: 3.25 MiB, allocs estimate: 82664.